In [ ]:

import numpy as np
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from io import BytesIO
import warnings

# Ignore warnings
warnings.filterwarnings('ignore')

# Initialize variables
E_voltages = None
input_widgets = None

# Create an Output widget for displaying dynamic content
output_box = widgets.Output()

# Function to create and display the input grid using ipywidgets
def create_input_grid(rows, cols, prefilled_data):
    global E_voltages, input_widgets

    # Initialize empty array for E_voltages
    E_voltages = np.zeros((rows, cols))

    # If prefilled_data is provided, update charges with it
    if prefilled_data is not None and prefilled_data.shape == (rows, cols):
        E_voltages = prefilled_data
        
    # Create a list to hold the FloatText widgets
    input_widgets = []

    # Clear the output box before displaying new content
    with output_box:
        clear_output(wait=True)
        
    # Create FloatText widgets and store them in input_widgets
    for i in range(rows):
        row_widgets = []
        for j in range(cols):
            widget = widgets.FloatText(value=E_voltages[i, j], description=f'[{rows - 1 - i}, {j}]')
            input_widgets.append(widget)
            row_widgets.append(widget)
        display(widgets.HBox(row_widgets))

# Function to handle widget value changes and update E_voltages
def update_E_voltages(change):
    global E_voltages, input_widgets
    rows, cols = E_voltages.shape
    for i in range(rows):
        for j in range(cols):
            E_voltages[i, j] = input_widgets[(rows - 1 - i) * cols + j].value  # Reverse row index

# Function to handle file upload and process the Excel file
def on_file_upload(change):
    global E_voltages, input_widgets
    file_contents = excel_upload_button.value[0]['content']
    excel_file = BytesIO(file_contents)
    df = pd.read_excel(excel_file, header=None)  # Read without assuming headers

    # Convert to numeric and drop empty rows and columns
    df = df.apply(pd.to_numeric, errors='coerce').dropna(how='all').dropna(axis=1, how='all')

    # Ensure to handle any non-numeric headers or rows that were meant to be included
    if df.iloc[0].isnull().all():  # Check if the first row is entirely empty and drop if so
        df = df.drop(0)

    # Replace empty values with 0
    df = df.fillna(0)

    # Find non-empty indices to determine the relevant data range
    non_empty_indices = np.argwhere(~np.isnan(df.to_numpy()))
    start_row, start_col = non_empty_indices.min(axis=0)
    end_row, end_col = non_empty_indices.max(axis=0)
    E_voltages = df.iloc[start_row:end_row + 1, start_col:end_col + 1].to_numpy()

    # Clear previous input widgets and display updated grid
    with output_box:
        clear_output(wait=True)
        create_input_grid(E_voltages.shape[0], E_voltages.shape[1], prefilled_data=E_voltages)

# Function to update plots
def update_plots(b=None):
    global E_voltages

    if E_voltages is None:
        return  # Return if E_voltages is not initialized

    # Calculate the max of each 2x2 sub-array
    E_max = np.array([
            [E_voltages[i:i+2, j:j+2].max() for j in range(E_voltages.shape[1]-1)]
            for i in range(E_voltages.shape[0]-1)
    ])
    
    # Variables for the change in x and y
    delta_x = 1
    delta_y = 1

    # Calculate Ex and Ey components
    Ex = -1 * np.diff(E_voltages, axis=1) / delta_x
    Ey = -1 * np.diff(E_voltages, axis=0) / delta_y

    # Remove the Ex extra row and Ey extra column
    Ex = Ex[:-1:]
    Ey = Ey[:, :-1]

    # Calculate the magnitude based on Ex and Ey components
    E_magnitude = np.sqrt(Ex**2 + Ey**2)

    # Add a small epsilon value to avoid division by zero
    epsilon = 1e-10  # A very small value
    E_magnitude += epsilon

    # Calculate the unit vector of the Ex and Ey (Ex or Ey / E_magnitude)
    Ex_normalized = Ex / E_magnitude
    Ey_normalized = Ey / E_magnitude

    # Set the meshgrid for the plots
    x = np.arange(Ex.shape[1])
    y = np.arange(Ex.shape[0])
    X, Y = np.meshgrid(x, y)

    # Clear previous plots within the output box
    with output_box:
        clear_output(wait=True)

        # Set up figure and axes
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        fig.suptitle('Electric Field', fontsize=40)

        # First subplot (Unit Vector Field)
        axs[0].quiver(X, Y, Ex_normalized, Ey_normalized, color='b')
        axs[0].set_title('UNIT VECTOR', fontsize=17)
        axs[0].set_xlabel('x')
        axs[0].set_ylabel('y')

        # Adjust the plot limits for complete visibility
        axs[0].set_xlim(-1, X.max() + 1)
        axs[0].set_ylim(-1, Y.max() + 1)
        axs[0].grid(True)

        # Second subplot (Stream Field)
        stream = axs[1].streamplot(X, Y, Ex, Ey, density=0.5, linewidth=1.5, arrowsize=2, color=E_max, cmap='gist_heat_r', broken_streamlines=False)
        colorbar = plt.colorbar(stream.lines, ax=axs[1], shrink=.7)
        colorbar.set_label('Voltage', fontsize=12, weight='bold')
        axs[1].set_title('STREAM PLOT', fontsize=17)
        axs[1].set_xlabel('x')
        axs[1].set_ylabel('y')
        
        # Adjust the plot limits for complete visibility
        axs[1].set_xlim(0, X.max())
        axs[1].set_ylim(0, Y.max())
        axs[1].grid(True)

        plt.tight_layout()
        plt.show()

# Create widgets for user input and file upload
info_heading = widgets.HTML(value="""
                            <h1>Miami Dade College | Hialeah Campus</h1>
                            <h1>Electric Field Plotter (Voltages)</h1>
                            """)
info_body = widgets.HTML(value="""
                        <h3>This tool allows you to visualize electric fields based on a table of voltages.</h3>
                        <h3>Upload an Excel table with voltage values. Make sure to fill all the values, (3x3) or greater.</h3>
                        """)

excel_upload_button = widgets.FileUpload(accept='.xlsx', multiple=False, description="Upload Excel Table")
update_button = widgets.Button(description="Run")

# Function to handle update button click event
def on_generate_plot(b):
    update_plots()

# Attach button click events
excel_upload_button.observe(on_file_upload, names='value')
update_button.on_click(on_generate_plot)

# Display initial widgets including the output box
display(info_heading, info_body, widgets.HBox([excel_upload_button, update_button]), output_box)

In [ ]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from io import BytesIO
import warnings

# Ignore warnings
warnings.filterwarnings('ignore')

# Initialize variables
charges_1 = None
input_widgets_1 = []
uploaded_file_1 = None

# Coulomb's constant
k_1 = 8.99e9
# A very small value for avoiding zero division
epsilon_1 = 1e-10

# Create an Output widget for displaying dynamic content
electric_field_output_1 = widgets.Output()

# Function to create and display the input table using ipywidgets
def create_input_grid_1(rows, cols, prefilled_data=None):
    global charges_1, input_widgets_1

    # Initialize charges
    charges_1 = np.zeros((rows, cols))

    # If prefilled_data is provided, update charges with it
    if prefilled_data is not None and prefilled_data.shape == (rows, cols):
        charges_1 = prefilled_data

    # Create a list to hold the FloatText widgets
    input_widgets_1 = []

    # Clear the electric field output box before displaying new content
    with electric_field_output_1:
        clear_output(wait=True)

        # Create FloatText widgets and store them in input_widgets
        for i in range(rows):
            row_widgets = []
            for j in range(cols):
                value = charges_1[i, j] if prefilled_data is not None else 0.0
                widget = widgets.FloatText(value=value, description=f'[{rows - 1 - i}, {j}]')
                input_widgets_1.append(widget)
                row_widgets.append(widget)
            display(widgets.HBox(row_widgets))

# Function to handle widget value changes and update charges
def update_charges_1():
    global charges_1, input_widgets_1
    rows, cols = charges_1.shape
    for i in range(rows):
        for j in range(cols):
            current_cell = (rows - 1 - i) * cols + j  # Reverse row index
            try:
                charges_1[i, j] = float(input_widgets_1[current_cell].value)
            except ValueError:
                charges_1[i, j] = 0.0

# Function to compute the electric field components due to a single charge
def electric_field_1(x, y, x0, y0, charge):
    q = charge
    dx = x - x0
    dy = y - y0
    r = np.sqrt(dx**2 + dy**2)
    if r > 0:  # Avoid division by zero
        Ex = k_1 * q * dx / r**3
        Ey = k_1 * q * dy / r**3
        return Ex, Ey
    else:
        return 0, 0

# Function to compute the potential due to a single charge
def potential_1(x, y, x0, y0, charge):
    q = charge
    dx = x - x0
    dy = y - y0
    r = np.sqrt(dx**2 + dy**2)
    if r > 0:
        return k_1 * q / r
    else:
        return 0

# Function to calculate the density for streamplot based on number of non-zero charges
def calculate_density_1(num_points):
    if num_points >= 5:
        return 0.5  # Base density for 5 points or more
    else:
        base_density = 0.2  # Base density for 1 point
        increment = 0.1     # Increment per additional point
        return base_density + (num_points - 1) * increment

# Function to update and display plots
def update_plots_1(b=None):
    global charges_1

    if charges_1 is None:
        return

    # Dimensions of the grid
    rows, cols = charges_1.shape

    # Create a mesh grid for the quiver plotting
    X, Y = np.meshgrid(np.arange(cols), np.arange(rows))
    Ex = np.zeros((rows, cols))
    Ey = np.zeros((rows, cols))

    finer = 100  # Number of points for the stream plot
    # Create a finely spaced mesh grid for the stream plot based on the (finer) int
    x_finer = np.linspace(-1, cols, finer)
    y_finer = np.linspace(-1, rows, finer)
    X_finer, Y_finer = np.meshgrid(x_finer, y_finer)

    # Initialize electric field components and potential for the finer spaced grid
    Ex_finer = np.zeros((finer, finer))
    Ey_finer = np.zeros((finer, finer))
    # Array to store the potential contributions for each charge
    V_finer = np.zeros((finer, finer))

    # Calculate the electric field components and potential at each grid point
    for y0 in range(rows):  # Iterate through each charge
        for x0 in range(cols):
            if charges_1[y0, x0] != 0:  # Consider only non-zero charges
                q = charges_1[y0, x0]
                for y in range(rows):  # Iterate through all grid points
                    for x in range(cols):
                        if (y == y0 and x == x0):
                            continue  # Skip the charge itself
                        Ex_val, Ey_val = electric_field_1(x, y, x0, y0, q)
                        Ex[y, x] += Ex_val
                        Ey[y, x] += Ey_val

                # Calculate electric field and potential contributions for the finer grid
                for j in range(finer):
                    for i in range(finer):
                        Ex_finer_val, Ey_finer_val = electric_field_1(X_finer[j, i], Y_finer[j, i], x0, y0, q)
                        Ex_finer[j, i] += Ex_finer_val
                        Ey_finer[j, i] += Ey_finer_val
                        V_finer[j, i] += potential_1(X_finer[j, i], Y_finer[j, i], x0, y0, q)

    # Using statistical methods to identify and replace outliers in V_finer
    mean = np.mean(V_finer)
    std_dev = np.std(V_finer)
    outliers = np.abs(V_finer - mean) > 2 * std_dev
    V_finer_no_outliers = np.where(outliers, mean, V_finer)

    # Calculate the magnitude of the electric field
    E_magnitude = np.sqrt(Ex**2 + Ey**2)
    E_magnitude += epsilon_1  # Avoid division by zero
    cmap = 'cool'  # Default colormap for mixed charges
    if np.all(charges_1 >= 0):
        cmap = 'autumn'
    elif np.all(charges_1 <= 0):
        cmap = 'winter'

    # Normalize electric field components
    Ex_normalized = Ex / E_magnitude
    Ey_normalized = Ey / E_magnitude

    # Clear the electric field output box before displaying new plots
    with electric_field_output_1:
        clear_output(wait=True)

        # Set up figure and axes
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        fig.suptitle('Electric Field', fontsize=20)

        # First subplot (Quiver Vector Field)
        axs[0].quiver(X, Y, Ex_normalized, Ey_normalized, color='blue')
        axs[0].set_title('QUIVER', fontsize=17)
        axs[0].set_xlabel('x')
        axs[0].set_ylabel('y')
        axs[0].set_xlim(-1, X.max() + 1)
        axs[0].set_ylim(-1, Y.max() + 1)
        axs[0].grid(True)

        # Second subplot (Stream Field)
        stream = axs[1].streamplot(X_finer, Y_finer, Ex_finer, Ey_finer,
                                   density=calculate_density_1(np.count_nonzero(charges_1)), linewidth=1.5,
                                   arrowsize=2, color=V_finer_no_outliers, cmap=cmap,
                                   integration_direction='both', broken_streamlines=False)

        # Include equipotential lines if checkbox is checked
        if include_contour_checkbox_1.value:
            V_levels = np.linspace(V_finer_no_outliers.min(), V_finer_no_outliers.max(), 10)
            axs[1].contour(X_finer, Y_finer, V_finer, levels=V_levels, colors='gray')

        axs[1].set_title('STREAM', fontsize=17)
        axs[1].set_xlabel('x')
        axs[1].set_ylabel('y')
        axs[1].set_xlim(-1, X.max() + 1)
        axs[1].set_ylim(-1, Y.max() + 1)
        axs[1].grid(True)

        plt.tight_layout()
        plt.show()

# Function to handle file upload and process the Excel file
def on_file_upload_1(change):
    global charges_1, uploaded_file_1
    file_contents = excel_upload_button_1.value[0]['content']
    excel_file = BytesIO(file_contents)
    df = pd.read_excel(excel_file, header=None)  # Read without assuming headers

    # Convert to numeric and drop NaN rows and columns
    df = df.apply(pd.to_numeric, errors='coerce').dropna(how='all').dropna(axis=1, how='all')

    # Ensure to handle any non-numeric headers or rows that were meant to be included
    if df.iloc[0].isnull().all():  # Check if the first row is entirely NaN and drop if so
        df = df.drop(0) 
        
    # Replace empty values with 0
    df = df.fillna(0)

    # Find non-empty indices to determine the relevant data range
    non_empty_indices = np.argwhere(~np.isnan(df.to_numpy()))
    start_row, start_col = non_empty_indices.min(axis=0)
    end_row, end_col = non_empty_indices.max(axis=0)
    charges_1 = df.iloc[start_row:end_row + 1, start_col:end_col + 1].to_numpy()

    clear_output(wait=True)
    create_input_grid_1(charges_1.shape[0], charges_1.shape[1], prefilled_data=charges_1)

    excel_upload_button_1._counter = 0

# Create widgets for user input
info_heading_1 = widgets.HTML(value="<h1>Electric Field Plotter (Charges)</h1>")
info_body_1 = widgets.HTML(value="""<div style="line-height: 1.2;">
<h3 style="margin-bottom: 5px;">This tool allows you to visualize electric fields based on a grid of charges (in Coulombs).</h3>
<h3><ul style="margin-top: 0px; margin-bottom: 10px;">
<li>Upload an Excel file with the charge values</li>
<li>Or create a table and input the values</li>
</ul></h3>
</div>""")

# Create a loading message
loading_message_1 = widgets.HTML("""
    <div style="font-size: 16px; color: #555; margin-top: 10px;">
        Loading...
    </div>
""")

# Define the buttons/widgets to use
table_button_1 = widgets.Button(description="Create Table")
excel_upload_button_1 = widgets.FileUpload(accept='.xlsx', multiple=False, description="Excel Upload")
submit_button_1 = widgets.Button(description="Run")
rows_widget_1 = widgets.IntSlider(value=5, min=3, max=15, step=1, description='Rows:')
cols_widget_1 = widgets.IntSlider(value=5, min=3, max=15, step=1, description='Columns:')
include_contour_checkbox_1 = widgets.Checkbox(value=True, description='Include Equipotential Lines')

# Display initial widgets using a VBox for the first row
widget_row1_1 = widgets.VBox([info_heading_1, info_body_1, widgets.HBox([table_button_1, rows_widget_1, cols_widget_1])])
widget_row2_1 = widgets.HBox([excel_upload_button_1, include_contour_checkbox_1, submit_button_1])

# Function to handle generate button click event
def on_generate_table_1(b):
    with electric_field_output_1:
        clear_output(wait=True)
        display(widget_row1_1, widget_row2_1)
        create_input_grid_1(rows_widget_1.value, cols_widget_1.value)

# Function to handle submit button click event
def on_submit_button_1(b):
    # Check if no table has been created, and display a message
    if charges_1 is None:
        with electric_field_output_1:
            clear_output(wait=True)
            print("Upload an Excel File or Create a Table.")
        return

    update_charges_1()

    # Check if all charges are zero, and display a message
    if np.all(charges_1 == 0):
        with electric_field_output_1:
            clear_output(wait=True)
            print("All charges are zero. Please enter a charge.")
        return

    with electric_field_output_1:
        clear_output(wait=True)
        display(loading_message_1)
    update_plots_1()

# Attach button click events
table_button_1.on_click(on_generate_table_1)
submit_button_1.on_click(on_submit_button_1)
excel_upload_button_1.observe(on_file_upload_1, names='value')

# Initial display of widgets
display(widget_row1_1, widget_row2_1, electric_field_output_1)